# Cvičenie 6: Monte Carlo metódy

Na predošlých dvoch cvičeniach sme videli, ako fungujú metódy Policy Iteration a Value Iteration. Na dnešnom cvičení sa pozrieme na Monte Carlo metódy, konkrétne no on-policy first-visit Monte Carlo algoritmus odhadu $\pi^*$. Pracovať pritom budeme s rovnakým svetom gridworld.

## Metóda Monte Carlo

Na rozdiel od predošlých dvoch metód založených na dynamickom programovaní, Monte Carlo metódy sa učia z priamych interakcií s prostredím, práve preto ich priebeh závisí od konkrétnych epizódach interakcie. Práve preto tieto metódy vieme aplikovať aj vtedy, ak nepoznáme úplne model, teda prechody medzi stavmi nám nie sú známe, vieme ich ale nasimulovať. Algoritmus zároveň pracuje explicitne s $\varepsilon$-soft politikou, ktorej príkladom je aj $\varepsilon$-greedy politika, teda akcia s najväčšou očakávanou hodnotou sa vyberie najčastejšie, ale s istou pravdepodobnosťou bude výber akcie náhodný.

Pseudokód algoritmu nájdete na obrázku nižšie, prediskutujte jeho kroky a význam premenných a parametrov, s ktorými algoritmus pracuje.

<img src="sources/lab06/monte_carlo.jpg" width="600">
<p style="text-align: center;">Zdroj: Sutton-Barto: Reinforcement Learning, 2nd ed., 2018</p>

Následne algoritmus aplikujeme pri riešení už známeho príkladu gridworld:

<img src="sources/lab06/gridworld_example2.jpg" width="300">

Príklad predstavuje svet *3x3* s cieľovou pozíciou v pravom hornom rohu, a s jednou pascou v druhom riadku a prvom stĺpci. K dispozícii sú štyri akcie: posun na sever, východ, juh a západ. Hráč nemôže prejsť cez stenu, v tomto prípade ostane na rovnakej pozícii, ale berie sa, akokeby vykonal krok. Ak hráč sa dostane do cieľa, obdrží odmenu 10, ak spadne do pasce, tak -10. V oboch prípadoch sa hra skončí. Pre ostatné kroky dostane odmenu -1.

Discount factor $\gamma = 0.8$. Prvý odhad hodnoty pre každú dvojicu stav-akcia bude 0: $Q(s, a) = 0 \ \ \  \forall s \in \mathcal{S}, a \in \mathcal{A}$.

## Ukážka fungovania algoritmu

Majme epizódu:

$s_{32}, E, -1, s_{33}, E, -1, s_{33}, E, -1, s_{33}, E, -1, s_{33}, E, -1, s_{33}, W, -1, s_{32}, E, -1, s_{33}, N, -1, s_{23}, N, 10$,

ktorá predstavuje úspešnú interakciu, keďže agent sa dostal na cieľovú pozíciu.

Pri náhodnej politike ($p(a|s) = 0,25 \ \  \forall s \in \mathcal{S}, a \in \mathcal{A}$) a s hodnotami $\varepsilon = 0,1$  a $G=0$ sa táto epizóda spracuje nasledovne.

### Krok $s_{23}, N, 10$

Vyhodnotenie začneme od posledného kroku. Najprv sa vypočíta nová hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot 0 + 10 = 10$

Ďalej zistíme, či kombinácia stavu a akcie $s_{23}, N$ sa nachádza v predošlých krokoch epizódy. Keďže takýto krok nebol vykonaný skôr v epizóde, urobíme ďalšie zmeny v hodnotách premenných. Najprv si zapamätáme skúsenosť:

$Returns(s_{23}, N) = \{ 10 \}$

Potom aktualizujeme odhad hodnoty dvojice stav-akcia:

$Q(s_{23}, N) = \frac{\sum Returns(s_{23}, N)}{|Returns(s_{23}, N)|}$

$Q(s_{23}) = \{ 10; 0; 0; 0 \}$

Akcia s najväčšou očakávanou hodnotou je teda $N$, na základe čoho aktualizujeme politiku pre daný stav:

$\pi(N|s_{23}) = 1 - \varepsilon + \frac{\varepsilon}{|\mathcal{A}|} = 1 - 0.1 + \frac{0.1}{4} = 0.925$

$\pi(E|s_{23}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

$\pi(S|s_{23}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

$\pi(W|s_{23}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

### Krok $s_{33}, N, -1$

Pokračujeme predchádzajúcim krokom epizódy. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot 10 - 1 = 7$

Znova zisťujeme, či kombinácia stavu a akcie $s_{33}, N$ sa nachádza v predošlých krokoch epizódy. Keďže takýto krok nebol vykonaný skôr v epizóde, urobíme ďalšie zmeny v hodnotách premenných. Najprv si zapamätáme skúsenosť:

$Returns(s_{33}, N) = \{ 7 \}$

Potom aktualizujeme odhad hodnoty dvojice stav-akcia:

$Q(s_{33}, N) = \frac{\sum Returns(s_{33}, N)}{|Returns(s_{33}, N)|}$

$Q(s_{33}) = \{ 7; 0; 0; 0 \}$

Akcia s najväčšou očakávanou hodnotou je teda $N$, na základe čoho aktualizujeme politiku pre daný stav:

$\pi(N|s_{33}) = 1 - \varepsilon + \frac{\varepsilon}{|\mathcal{A}|} = 1 - 0.1 + \frac{0.1}{4} = 0.925$

$\pi(E|s_{33}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

$\pi(S|s_{33}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

$\pi(W|s_{33}) = \frac{\varepsilon}{|\mathcal{A}|} = \frac{0.1}{4} = 0.025$

### Krok $s_{32}, E, -1$

Pokračujeme predchádzajúcim krokom epizódy. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot 7 - 1 = 4,6$

Znova zisťujeme, či kombinácia stavu a akcie $s_{32}, E$ sa nachádza v predošlých krokoch epizódy. Vidíme, že presne ten istý krok bol vykonaný ako prvý v epizóde, takže ďalšie zmeny pre tento krok neurobíme.

### Krok $s_{33}, W, -1$

Pokračujeme predchádzajúcim krokom epizódy. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot 4,6 - 1 = 2,68$

Keďže krok $s_{33}, W$ nebol vykonaný skôr v epizóde, urobíme ďalšie zmeny v hodnotách premenných. Najprv si zapamätáme skúsenosť:

$Returns(s_{33}, W) = \{ 2,68 \}$

Potom aktualizujeme odhad hodnoty dvojice stav-akcia:

$Q(s_{33}) = \{ 7; 0; 0; 2,68 \}$

Akcia s najväčšou očakávanou hodnotou ostáva $N$.

### Krok $s_{33}, E, -1$

Pokračujeme predchádzajúcim krokom epizódy. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot 2,68 - 1 = 1,144$

Takýto krok bol vykonaný hneď niekoľkokrát za sebou, takže túto aktualizáciu urobíme až po prvý výskyt tejto kombinácie v epizóde. Konečná hodnota je potom $G = -1,854272$. Je to až táto skúsenosť, ktorú si zapamätáme (first-visit):

$Returns(s_{33}, E) = \{ -1,854272 \}$

Z toho:

$Q(s_{33}, E) = \{ 7; -1,854272; 0; 2,68 \}$

Najlepšia akcia je stále $N$.

### Krok $s_{32}, E, -1$

Ostáva nám už len prvý krok epizódy. Najprv sa aktualizuje hodnota $G$:

$G \leftarrow \gamma \cdot G + R = 0.8 \cdot (-1,854272) - 1 = -2,4834176$

Skúsenosť si zapamätáme skúsenosť, a aktualizujeme ostatné hodnoty:

$Returns(s_{32}, E) = \{ -2,4834176 \}$

$Q(s_{32}) = \{ -2,4834176; 0; 0; 0 \}$

Akcia s najväčšou očakávanou hodnotou sa vyberie náhodne z množiny $\{ N, S, W \}$.

## Úloha:

Vypočítajte aktualizované hodnoty pomocou metódy Monte Carlo na nasledujúcich epizódach (svet je rovnaký):

$s_{23}, S, -1, s_{33}, W, -1, s_{32}, S, -1, s_{32}, E, -1, s_{33}, E, -1, s_{33}, W, -1, s_{32}, E, -1, s_{33}, W, -1, s_{32}, E, -1, s_{33}, W, -1, s_{32}, W, -1, s_{31}, E, -1, s_{32}, N, -1, s_{22}, N, -1, s_{12}, W, -1, s_{11}, W, -1, s_{11}, N, -1, s_{11}, E, -1, s_{12}, E, 10$

$s_{23}, W, -1, s_{22}, W, -10$

$s_{11}, E, -1, s_{12}, E, 10$

Vaše riešenia si skontrolujte pomocou [ukážkovej implementácie metódy](sources/lab06/monte_carlo.py). Existuje možnosť, že metóda nájde optimálne riešenie už po porvej epizóde?

## Domáca úloha

Na základe ukážkovej implementácie zadefinujte vlastného Monte Carlo agenta, ktorý bude pracovať s environmentom, ktorý sme implementovali na druhom cvičení. [Ukážkový environment nájdete tu.](sources/lab02/gridworld.py)

Agenta natrénujte na svete minimálne *6x6* s jednou cieľovou pozíciou a s minimálne jednou pascou. Do sveta pridajte aj tri steny, ale dávajte si pozor na to, aby existovala cesta z každej pozície do cieľa. Vaše riešenie s Monte Carlo agentom (iba jeden Python skript) cez MS Teams. Skript nech obsahuje aj trénovanie na svete (implementáciu gridworldu nemusíte odovzdať). [Kostru riešenia nájdete tu](sources/lab06/monte_carlo_agent.py), no môžete použiť aj vlastný prístup.

**Poznámka**: Dávajte si pozor na odlišnú reprezentáciu stavov v environmente Gridworld. V našej implementácii je každý stav reprezentovaný dvojicou hodnotou (ako na obrázku vyššie). Horný ľavý roh je pozícia (0, 0), prvá hodnota udáva riadok, a druhá hodnota určuje stĺpec. Pre kontrolu sveta môžete použiť metódu `render` v triede `Gridworld`.